In [1]:
import pandas as pd
import time
import numpy as np
import pickle
#from ClusteringPredictiveModel import ClusteringPredictiveModel
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from  matplotlib import pyplot
#from sklearn.ensemble import RandomForestClassifier
#from LastStateEncoder import LastStateEncoder
from datetime import datetime, timedelta

# pre-processing data

In [3]:
data=pd.read_csv('BPIC 2017 just complete cases.csv',sep=';')

In [13]:
def n_ev(x):
    x['event_nr']=[d for d in range(1,len(x)+1)]
    return x

In [14]:
data=data.groupby(['Case ID'],sort=False).apply(lambda x: n_ev(x))

In [5]:
#data.drop(other_col,axis=1,inplace=True)

In [16]:
data.to_csv('data_preprocessed.csv',sep=';')

# =========

In [17]:
data=pd.read_csv('data_preprocessed.csv',sep=';')

In [18]:


#global case_id_col,event_col,timestamp_col,label_col,pos_label,numeric_cols,meta_event_data,other_col,variant_col

case_id_col = "Case ID"
event_col =[ "Activity"]
timestamp_col = ["Complete Timestamp","Start Timestamp"]
label_col = 'lbl'
pos_label = 'A_Pending'

numeric_cols = [ '(case) RequestedAmount',label_col]  
meta_event_data=['Resource','FirstWithdrawalAmount', 'MonthlyCost', 'NumberOfTerms','OfferedAmount']

other_col= [ 'Variant index','EventOrigin', 'OfferID','Unnamed: 0','Transition','milli_seconds',
          'EventID','lifecycle:transition','days','D_seconds',
          'Accepted', 'Selected' ,  'outcome',
          'Variant', 'Action' ,"Complete Timestamp","Start Timestamp"]


static_cols=["Case ID",'lbl','(case) RequestedAmount','(case) LoanGoal','(case) ApplicationType']
dynamic_cols=[ "Activity",'Resource','FirstWithdrawalAmount', 'MonthlyCost', 'NumberOfTerms','OfferedAmount', 'CreditScore']
cat_cols = [ '(case) LoanGoal','(case) ApplicationType'] 

event_nr_col = "index"
text_col = "Activity"
pos_label = "A_Pending"

In [19]:
data=data[data['Case ID'].isin(data['Case ID'].unique()[0:30])]

In [20]:
from sklearn.preprocessing import LabelEncoder

def trace(x):
    ascii_offset=161
    l=''
    for ac in x['Activity']:
        l+=chr(int(ac)+ascii_offset)
    return l

def preprocessing (data_set):
    
    lb_make = LabelEncoder()
    data_set['Activity'] = lb_make.fit_transform(data_set['Activity'])

    # aggregate activities by case
    grouped = data_set.groupby(['Case ID'], sort=False)

    # extract  encoded trace from actvities
    Activities = grouped.apply(lambda x: trace(x) )
    
    label = data_set.groupby(['Case ID'],sort=False).first()['lbl']
    ac_lbl=pd.DataFrame(columns=['Case ID','Activity','lbl'])
    ac_lbl['Case ID']=Activities.index
    ac_lbl['Activity']=list(Activities)
    ac_lbl['lbl']=list(label)
    return ac_lbl

In [6]:
from TextTransformers import LDATransformer

In [7]:
lda=LDATransformer (num_topics=2, tfidf=False, 
                 passes=3, iterations=3, min_prob=0, min_freq=0, save_dict=False, dict_file=None, random_seed=None)

In [22]:
lda.fit(data)

In [15]:
lda.lda_model[lda._generate_corpus_data(data)]

In [23]:
lda.fit_transform(data)

,topic1_event1,topic2_event1,topic1_event2,topic2_event2,topic1_event3,topic2_event3,topic1_event4,topic2_event4,topic1_event5,topic2_event5,...,topic1_event9,topic2_event9,topic1_event10,topic2_event10,topic1_event11,topic2_event11,topic1_event12,topic2_event12,topic1_event13,topic2_event13
0,0.254267,0.745733,0.828676,0.171324,0.747310,0.252690,0.169558,0.830442,0.874207,0.125793,...,0.747649,0.252351,0.747655,0.252345,0.747654,0.252346,0.251536,0.748464,0.251030,0.748970
1,0.254293,0.745707,0.268042,0.731959,0.747309,0.252691,0.169544,0.830456,0.874207,0.125793,...,0.747650,0.252350,0.747646,0.252354,0.747654,0.252346,0.251537,0.748463,0.251091,0.748909
2,0.254302,0.745698,0.290844,0.709156,0.747316,0.252684,0.169592,0.830408,0.874207,0.125793,...,0.747652,0.252348,0.747656,0.252344,0.747645,0.252355,0.251537,0.748463,0.251036,0.748964
3,0.254303,0.745697,0.829845,0.170155,0.739743,0.260256,0.169567,0.830433,0.874204,0.125796,...,0.747650,0.252350,0.747649,0.252351,0.747649,0.252351,0.251536,0.748464,0.251078,0.748922
4,0.254274,0.745726,0.252207,0.747793,0.511876,0.488124,0.169581,0.830419,0.874208,0.125792,...,0.747653,0.252347,0.747655,0.252345,0.747652,0.252348,0.251535,0.748465,0.251054,0.748946
5,0.254295,0.745705,0.171431,0.828569,0.529049,0.470951,0.169549,0.830451,0.874209,0.125791,...,0.269874,0.730126,0.272019,0.727981,0.746062,0.253938,0.251535,0.748465,0.251072,0.748928
6,0.254286,0.745714,0.748925,0.251075,0.473170,0.526830,0.169587,0.830413,0.874205,0.125795,...,0.747648,0.252352,0.747652,0.252348,0.747647,0.252353,0.251536,0.748464,0.251060,0.748940
7,0.254299,0.745701,0.893795,0.106205,0.489421,0.510579,0.169527,0.830473,0.874204,0.125796,...,0.747654,0.252346,0.747653,0.252347,0.747654,0.252346,0.251537,0.748463,0.251099,0.748901
8,0.254309,0.745691,0.873273,0.126727,0.523982,0.476017,0.169574,0.830426,0.874205,0.125795,...,0.747651,0.252349,0.747650,0.252350,0.747646,0.252354,0.251537,0.748463,0.251077,0.748923
9,0.254293,0.745707,0.747101,0.252899,0.509490,0.490510,0.169580,0.830420,0.874206,0.125794,...,0.747656,0.252344,0.747655,0.252345,0.747652,0.252348,0.251537,0.748463,0.251029,0.748971


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.model_selection import StratifiedKFold
import sys
sys.path.append('..')
from PredictiveMonitor import PredictiveMonitor

#data_filepath = "BPIC 2017 just complete cases.csv"
#data = pd.read_csv(data_filepath, sep=";")    #, encoding="utf-8")


event_nr_col ='event_nr'
text_col = "Activity"
pos_label = "A_Pending"

# divide into train and test data
train_names, test_names = train_test_split( data[case_id_col].unique(), train_size = 4.0/5, random_state = 22 )
train = data[data[case_id_col].isin(train_names)]
test = data[data[case_id_col].isin(test_names)]
pre=preprocessing(train)
# create five folds for cross-validation out of training data
kf = StratifiedKFold( n_splits=5, shuffle=True, random_state=22)
kk=kf.split(pre[case_id_col],pre[label_col])


confidences = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95,0.97,0.98,0.99,0.999]
text_transformer_type = "PVTransformer"
cls_methods = ["rf", "logit"]

encoder_kwargs = {"event_nr_col":event_nr_col, "static_cols":static_cols, "dynamic_cols":dynamic_cols, "cat_cols":cat_cols,
                                      "oversample_fit":False, "minority_label":pos_label, "fillna":True,
                  "random_state":22}
 

C:\Users\MSI\Miniconda3\envs\ch\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
C:\Users\MSI\Miniconda3\envs\ch\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [ ]:
                                     

part = 1
for train_index, test_index in kk:
    
    # create train and validation data according to current fold
    current_train_names = train_names[train_index]
    train_chunk = train[train[case_id_col].isin(current_train_names)]
    train_chunk['index']=train_chunk.index
    current_test_names = train_names[test_index]
    test_chunk = train[train[case_id_col].isin(current_test_names)]
    test_chunk['index']=test_chunk.index

    
    for cls_method in cls_methods:
        if cls_method == "rf":
            cls_kwargs = {"n_estimators":500, "random_state":22}
        else:
            cls_kwargs = {"random_state":22}
            
        for size in [10, 25, 50, 75, 100, 200, 400]:
            for window in range(1, 13):
                    
                transformer_kwargs = {"size":size, "window":window, "random_seed":22, "epochs":10}

                # train
                predictive_monitor = PredictiveMonitor(event_nr_col=event_nr_col, case_id_col=case_id_col, label_col=label_col, pos_label=pos_label, encoder_kwargs=encoder_kwargs, cls_kwargs=cls_kwargs, transformer_kwargs=transformer_kwargs, text_col=text_col, text_transformer_type=text_transformer_type, cls_method=cls_method)
                predictive_monitor.train(train_chunk)

                # test
                predictive_monitor.test(test_chunk, confidences, evaluate=True, output_filename="cv_results/pv_size%s_window%s_%s_part%s"%(size, window, cls_method, part))

    part += 1

C:\Users\MSI\Miniconda3\envs\ch\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\MSI\Miniconda3\envs\ch\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\MSI\Miniconda3\envs\ch\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versu

dt_train                      Case ID  Activity  Resource (case) ApplicationType  \
0     Application_652823628         4    User_1             New credit   
1     Application_652823628         8    User_1             New credit   
2     Application_652823628         3    User_1             New credit   
3     Application_652823628        19   User_17             New credit   
4     Application_652823628         0   User_52             New credit   
5     Application_652823628        12   User_52             New credit   
6     Application_652823628        13   User_52             New credit   
7     Application_652823628        16   User_52             New credit   
8     Application_652823628        17   User_52             New credit   
9     Application_652823628         2   User_52             New credit   
10    Application_652823628        21  User_117             New credit   
11    Application_652823628         9  User_117             New credit   
12    Application_652823628 

In [ ]:
predictive_monitor.

In [12]:
 c[c.index == c.index.max()][static_cols]

,Case ID,lbl,(case) RequestedAmount,(case) LoanGoal,(case) ApplicationType
3,Application_652823628,A_Pending,20000.0,Existing loan takeover,New credit


In [8]:
c=data.head(4)

In [11]:
c

,Case ID,Activity,Resource,(case) ApplicationType,(case) LoanGoal,(case) RequestedAmount,CreditScore,FirstWithdrawalAmount,MonthlyCost,NumberOfTerms,OfferedAmount,lbl
0,Application_652823628,A_Create Application,User_1,New credit,Existing loan takeover,20000.0,NaN,NaN,NaN,NaN,NaN,A_Pending
1,Application_652823628,A_Submitted,User_1,New credit,Existing loan takeover,20000.0,NaN,NaN,NaN,NaN,NaN,A_Pending
2,Application_652823628,A_Concept,User_1,New credit,Existing loan takeover,20000.0,NaN,NaN,NaN,NaN,NaN,A_Pending
3,Application_652823628,W_Complete application,User_17,New credit,Existing loan takeover,20000.0,NaN,NaN,NaN,NaN,NaN,A_Pending


In [8]:
from PredictiveMonitor import PredictiveMonitor

encoder_kwargs = {"event_nr_col":event_nr_col, "static_cols":static_cols, "dynamic_cols":dynamic_cols,
                  "cat_cols":cat_cols, "oversample_fit":False, "minority_label":"A_Pending", 
                  "fillna":True, "random_state":22}
transformer_kwargs = {"ngram_max":4, "alpha":0.002, "nr_selected":11, 
                  "pos_label":pos_label}
cls_kwargs = {"n_estimators":500, "random_state":22}

predictive_monitor = PredictiveMonitor(event_nr_col=event_nr_col, case_id_col=case_id_col,
                                      label_col=label_col, pos_label=pos_label, text_col=text_col,
                                      text_transformer_type="NBLogCountRatioTransformer", cls_method="rf",
                                      encoder_kwargs=encoder_kwargs, transformer_kwargs=transformer_kwargs, 
                                      cls_kwargs=cls_kwargs)

predictive_monitor.train(train_chunk)
predictive_monitor.test(train_chunk, confidences=[0.5, 0.75, 0.9], evaluate=True, output_filename="example_output.txt")

dt_train                         Case ID  Activity  Resource (case) ApplicationType  \
0        Application_652823628         4    User_1             New credit   
1        Application_652823628         8    User_1             New credit   
2        Application_652823628         3    User_1             New credit   
3        Application_652823628        21   User_17             New credit   
4        Application_652823628         0   User_52             New credit   
5        Application_652823628        12   User_52             New credit   
6        Application_652823628        13   User_52             New credit   
7        Application_652823628        16   User_52             New credit   
8        Application_652823628        19   User_52             New credit   
9        Application_652823628         2   User_52             New credit   
10       Application_652823628        24  User_117             New credit   
11       Application_652823628         9  User_117             New

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [13]:
text_col

'Activity'

In [23]:
train

,Case ID,Activity,Resource,(case) ApplicationType,(case) LoanGoal,(case) RequestedAmount,CreditScore,FirstWithdrawalAmount,MonthlyCost,NumberOfTerms,OfferedAmount,lbl
0,Application_652823628,4,User_1,New credit,Existing loan takeover,20000.0,NaN,NaN,NaN,NaN,NaN,A_Pending
1,Application_652823628,8,User_1,New credit,Existing loan takeover,20000.0,NaN,NaN,NaN,NaN,NaN,A_Pending
2,Application_652823628,3,User_1,New credit,Existing loan takeover,20000.0,NaN,NaN,NaN,NaN,NaN,A_Pending
3,Application_652823628,21,User_17,New credit,Existing loan takeover,20000.0,NaN,NaN,NaN,NaN,NaN,A_Pending
4,Application_652823628,0,User_52,New credit,Existing loan takeover,20000.0,NaN,NaN,NaN,NaN,NaN,A_Pending
5,Application_652823628,12,User_52,New credit,Existing loan takeover,20000.0,979.0,20000.0,498.29,44.0,20000.0,A_Pending
6,Application_652823628,13,User_52,New credit,Existing loan takeover,20000.0,NaN,NaN,NaN,NaN,NaN,A_Pending
7,Application_652823628,16,User_52,New credit,Existing loan takeover,20000.0,NaN,NaN,NaN,NaN,NaN,A_Pending
8,Application_652823628,19,User_52,New credit,Existing loan takeover,20000.0,NaN,NaN,NaN,NaN,NaN,A_Pending
9,Application_652823628,2,User_52,New credit,Existing loan takeover,20000.0,NaN,NaN,NaN,NaN,NaN,A_Pending


In [25]:
fff=np.hstack(np.vsplit(train,4))

In [35]:
fff[0:5]

array([['Application_652823628', 4, 'User_1', 'New credit',
        'Existing loan takeover', 20000.0, nan, nan, nan, nan, nan,
        'A_Pending', 'Application_195141843', 0, 'User_32', 'New credit',
        'Remaining debt home', 15000.0, nan, nan, nan, nan, nan,
        'A_Pending', 'Application_1734458355', 0, 'User_23', 'New credit',
        'Existing loan takeover', 15000.0, nan, nan, nan, nan, nan,
        'A_Pending', 'Application_1738448869', 8, 'User_1', 'New credit',
        'Existing loan takeover', 24000.0, nan, nan, nan, nan, nan,
        'A_Pending'],
       ['Application_652823628', 8, 'User_1', 'New credit',
        'Existing loan takeover', 20000.0, nan, nan, nan, nan, nan,
        'A_Pending', 'Application_195141843', 12, 'User_32', 'New credit',
        'Remaining debt home', 15000.0, 924.0, 15000.0, 157.59, 120.0,
        15000.0, 'A_Pending', 'Application_1734458355', 12, 'User_23',
        'New credit', 'Existing loan takeover', 15000.0, 0.0, 0.0, 233.98,
      